# LLM with Web Search and Crawl

풍부한 컨텍스틀 활용하기 위해 구글검색 된 결과의 최상위 n개의 페이지를 크롤링하여 LLM에 제공하기 위한 코드입니다.



In [54]:
import re
import requests
import sys
import os
from openai import AzureOpenAI
import tiktoken
from dotenv import load_dotenv
load_dotenv(override=True)

client = AzureOpenAI(
  azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT"), 
  api_key=os.getenv("AZURE_OPENAI_KEY"),  
  api_version="2024-08-01-preview"
)

CHAT_COMPLETIONS_MODEL = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')

bs4 or scrapy?

In [57]:
import requests
import json
import scrapy
from bs4 import BeautifulSoup

GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_CSE_ID = os.getenv("GOOGLE_CSE_ID")

def extract_text_and_tables_by_bs4(url):

    response = requests.get(url)
    soup = BeautifulSoup(response.text, "html.parser")
    # Extract main text
    paragraphs = [p.get_text().strip() for p in soup.find_all("p") if p.get_text().strip()]
    text = "\n".join(paragraphs)
    
    return text

def extract_text_and_tables(url):

    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0.0.0 Safari/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=4)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Request failed: {e}")
        return ""

    selector = scrapy.Selector(text=response.text)
    paragraphs = [p.strip() for p in selector.css('p::text').getall() if p.strip()]
    text = "\n".join(paragraphs)

    return text

def google_search(query, num=5):
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "q": query,
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_CSE_ID,
        "num": num, 
        "locale": "ko",  # 한국어로 검색
    }

    response = requests.get(url, params=params)
    results = response.json()
    return results.get("items", [])


def add_context(top_urls = []):
    contexts = []
    for url in top_urls:
        text = extract_text_and_tables(url)
        contexts.append(text)

    return contexts   
       
QUERY_REWRITE_PROMPT = """
        너는 구글 검색과 LLM을 위한 질문 재작성 전문가야. 사용자가 입력한 질문을 두 가지 목적에 맞게 재작성해줘.

        Google Search용 Query Rewrite 프롬프트
                사용자의 질문을 실제 구글 검색창에 입력할 수 있도록, 명확하고 간결한 검색어로 재작성해줘. 불필요한 문장이나 맥락 설명은 빼고, 핵심 키워드와 관련 정보를 중심으로 검색에 최적화된 형태로 만들어줘.
        LLM Query용 Rewrite 프롬프트
                사용자의 질문을 LLM이 더 잘 이해하고 답변할 수 있도록, 맥락과 의도를 명확히 드러내는 자연스러운 문장으로 재작성해줘. 필요한 경우, 추가 설명이나 세부 조건을 포함해서 질문의 목적이 분명히 드러나도록 만들어줘.

        예시:   
        * 질문: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
        * 구글 검색용 재작성: 삼성전자 3구 이상 인덕션 추천
        * LLM 답변용 재작성: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.

        출력포맷. 반드시 json 형식으로 출력해줘.:
        {"google_search": "구글 검색용 재작성", "llm_query": "LLM 답변용 재작성"}
        """     
  
def rewrite_query_for_search_and_llm(query, client: AzureOpenAI):
        response = client.chat.completions.create(
            model=CHAT_COMPLETIONS_MODEL,
            messages=[
                {"role": "system", "content": QUERY_REWRITE_PROMPT},
                {"role": "user", "content": query}
            ],
            temperature=0.8,
            max_tokens=300,
            response_format= {"type": "json_object"},
        )
        
        return json.loads(response.choices[0].message.content.strip())







In [58]:
from IPython.display import Markdown, display
from datetime import datetime


RESULTS_COUNT = 5

inputs = [
            "삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘",
            "부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘",
            "삼성전자 25년 제품이 작년 대비 좋아진것은",
            "삼성전자 JBL과 하만카돈 차이점이 뭐야",
            "갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요",
            "삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나"
    ]   

# inputs = ["삼성전자 25년 제품이 작년 대비 좋아진것은"]

for input in inputs:
    print(f"Original Input: {input}")
    
    query_rewrite = rewrite_query_for_search_and_llm(input, client)
    print(f"Google Search Query: {query_rewrite['google_search']}")
    print(f"LLM Query: {query_rewrite['llm_query']}")

    results = google_search(query_rewrite['google_search'], RESULTS_COUNT)
    print(f"Google Search Results: {len(results)}")
    top_urls = [results[i]["link"] for i in range(RESULTS_COUNT)]

    contexts = add_context(top_urls)

    # for i, context in enumerate(contexts):
    #     print(f"Context {i+1}: {context[:1000]}...")  # Print first 1000 chars of each context
    #     print("\n--- End of Context ---\n")

    now = datetime.now()
    year = now.year
    month = now.month
    day = now.day

    system_prompt="너는 삼성전자 제품 관련 정보를 제공하는 챗봇이야. 답변은 마크다운으로 작성해줘."
    user_prompt=f"""
            너는 아래 제공하는 구글에서 검색한 컨텍스트를 바탕으로 질문에 대한 답변을 제공해야 해. 컨텍스트를 최대한 활용하여 풍부하게 답변을 해야해. 
            현재는 {year}년 {month}월 {day}일이므로 최신의 데이터를 기반으로 답변을 해줘.

            구글에서 제공한 컨텍스트: {contexts}
            질문: {query_rewrite['llm_query']}
            """

    response = client.chat.completions.create(
        model=CHAT_COMPLETIONS_MODEL,
        messages = [{"role":"system", "content":system_prompt},
                    {"role":"user","content": user_prompt}],
        top_p=0.9,
        max_tokens=1500
    )

    display(Markdown(response.choices[0].message.content))



Original Input: 삼성전자 제품 중 2구 말고 다른 인덕션 추천해줘
Google Search Query: 삼성전자 3구 이상 인덕션 추천
LLM Query: 삼성전자 인덕션 중 2구 모델이 아닌, 3구 이상 또는 다양한 화구 수를 가진 다른 인덕션 제품을 추천해 주세요. 각 모델의 주요 기능과 장점도 함께 알려주세요.
Google Search Results: 5


삼성전자에서 제공하는 다양한 화구 수를 가진 인덕션 모델 중 3구 이상 제품에 대해 소개해 드리겠습니다. 각 모델은 다양한 기능과 장점을 가지고 있어 소비자의 다양한 요구를 충족시킬 수 있습니다.

### 삼성전자 비스포크 AI 3구 인덕션 NZ63DB607CF
- **출력**: 대화구 3,400W, 중화구 2,600W, 소화구 1,200W로 최상급의 화력을 자랑합니다.
- **편의성**: AI 넘침 방지 모드와 저소음 기능(DCN) 탑재, 조작부는 심플하고 대화형 알림창을 통해 사용자 친화적입니다.
- **디자인**: 프레임리스 마감으로 심플하며, 두께가 얇아져서 설치 시 공간 활용이 용이합니다.
- **가격**: 최상급 기능과 화력에도 불구하고 가격은 100만 원 미만으로 매우 경쟁력 있는 가성비를 제공합니다.

### 삼성전자 비스포크 AI 3구 인덕션 NZ63DB657CA*S
- **출력**: 모든 화구의 일반 출력이 2,000W를 넘어 최상급 화력을 보장합니다.
- **편의성**: LCD 대화형 알림창, DCN 기능, AI 끓음 감지 기능 및 앱 연동으로 외부에서도 제어 가능하여 편리합니다.
- **디자인**: 이전 모델 대비 얇아진 두께와 촘촘한 4분할 플렉스 존으로 다양한 크기의 용기를 사용할 수 있습니다.
- **가격**: 프리미엄 인덕션으로서 성능과 가격 면에서 합리적인 선택을 제공합니다.

### 삼성전자 NZ63T4507AK (하이브리드 인덕션)
- **출력**: 인덕션 대화구 출력은 2,300W, 중화구 하이라이트는 직경 20cm로 넓은 용기에 적합합니다.
- **화구 구성**: 대화구는 인덕션, 나머지 화구는 하이라이트로 구성되어 다양한 용기 사용이 가능합니다.
- **상판**: 독일 SCHOTT사의 상판을 사용하여 내구성이 우수합니다.

각 모델은 삼성전자의 우수한 기술력과 사용자의 편의를 고려한 기능들을 탑재하고 있어, 다양한 요구와 예산에 맞춰 선택할 수 있습니다. 화구 구성과 각종 편의 기능을 고려하여 구매를 결정하시면 좋습니다.

Original Input: 부모님에게 선물하고 싶은데 삼성전자 TV 추천해줘
Google Search Query: 삼성전자 TV 부모님 선물 추천
LLM Query: 부모님께 선물할 만한 삼성전자 TV를 추천해 주세요. 모델별 특징과 장점을 함께 알려주시면 감사하겠습니다.
Google Search Results: 5
Request failed: HTTPSConnectionPool(host='news.samsung.com', port=443): Read timed out. (read timeout=4)
Request failed: HTTPSConnectionPool(host='coolenjoy.net', port=443): Read timed out. (read timeout=4)


부모님께 선물할 만한 삼성전자 TV를 추천드리기 위해 현재의 요구와 예산을 고려하여 몇 가지 모델을 제안해드립니다.

### 삼성전자 TV 추천 모델

1. **삼성 QLED 75Q60C**
   - **특징**: 이 모델은 삼성의 QLED 기술을 사용하여 뛰어난 색 재현과 명암비를 제공합니다. 크기가 75인치로, 대형 화면을 원하는 경우 적합합니다.
   - **장점**: 
     - 뛰어난 화질: QLED 기술은 풍부한 색감과 높은 명암비를 제공합니다.
     - 스마트 기능: Samsung Smart Hub를 통해 다양한 콘텐츠를 쉽게 탐색할 수 있습니다.
     - 슬림한 디자인: 모던한 디자인으로 공간을 세련되게 만들어 줍니다.

2. **삼성 Crystal UHD 75CU7000**
   - **특징**: Crystal UHD 기술을 통해 선명한 화질과 자연스러운 색감을 제공합니다. 
   - **장점**: 
     - 가격 대비 성능: 합리적인 가격으로 고품질의 UHD 화질을 즐길 수 있습니다.
     - 다양한 연결 옵션: 여러 HDMI 포트 및 USB 포트를 통해 다양한 기기를 연결할 수 있습니다.
     - 에코 모드: 에너지 절약 모드를 지원하여 전력 소비를 줄일 수 있습니다.

3. **삼성 Neo QLED 75QN85B**
   - **특징**: Neo QLED는 미니 LED 백라이트 기술을 사용하여 더 깊은 명암비와 더 정밀한 밝기 조절을 제공합니다.
   - **장점**:
     - 탁월한 화질: Neo QLED 기술로 인해 디테일한 화질과 깊이 있는 블랙을 표현합니다.
     - AI 기능: AI 기반 화질 개선 기능을 통해 시청하는 콘텐츠에 맞춰 최적의 화면을 제공합니다.
     - 음향 시스템: 강력한 내장 스피커 시스템으로 몰입감을 높이는 사운드를 제공합니다.

### 추가 팁

- **브랜드 신뢰도**: 부모님 세대에서는 LG나 삼성과 같은 대기업 브랜드를 선호하는 경향이 있습니다. 이들 브랜드는 A/S가 잘 되어 있으며, 사용 중 문제가 발생하더라도 빠르게 해결할 수 있습니다.
- **할인 시기**: 연말이나 블랙 프라이데이 같은 할인 시즌을 활용하면 더욱 좋은 가격으로 구매할 수 있습니다.

이 모델들은 부모님께서 TV를 통해 다양한 콘텐츠를 즐기시기에 적합하며, 삼성의 기술력이 담긴 다양한 기능을 제공하여 만족스러운 사용 경험을 선사할 것입니다.

Original Input: 삼성전자 25년 제품이 작년 대비 좋아진것은
Google Search Query: 삼성전자 2025년 제품 개선점
LLM Query: 삼성전자의 2025년 제품에서 2024년 대비 어떤 점이 개선되었는지 알려주세요. 기능, 성능, 디자인 등의 측면에서 비교 부탁드립니다.
Google Search Results: 5
Request failed: HTTPSConnectionPool(host='news.samsung.com', port=443): Read timed out. (read timeout=4)
Request failed: HTTPSConnectionPool(host='news.samsung.com', port=443): Read timed out. (read timeout=4)


...

삼성전자는 오디오 및 커넥티드 기술 분야의 글로벌 리더인 하만 인터내셔널을 소유하고 있으며, 하만 인터내셔널은 여러 유명 오디오 브랜드를 운영합니다. 그 중 두 가지 주요 브랜드는 JBL과 하만카돈입니다. 이 두 브랜드는 각기 다른 시장과 고객층을 대상으로 독특한 제품 라인업을 제공합니다.

### JBL
1. **주요 특징**: 
   - JBL은 강력한 사운드와 내구성 있는 제품을 강조하는 브랜드입니다. 특히 야외 활동이나 스포츠를 위한 포터블 스피커와 헤드폰으로 잘 알려져 있습니다.
   - JBL 제품은 일반적으로 대중 시장을 대상으로 하며, 다양한 색상과 스타일의 디자인을 제공합니다.
   - JBL의 사운드 프로파일은 베이스 중심으로, 파워풀하고 생동감 있는 음악 재생을 특징으로 합니다.

2. **제품 라인업**:
   - **포터블 스피커**: JBL Charge, JBL Flip, JBL Xtreme 등 다양한 모델이 있으며, 방수 기능과 긴 배터리 수명을 제공합니다.
   - **헤드폰 및 이어버드**: JBL Tune, JBL Reflect 시리즈는 스포츠 및 일상 생활에서 사용할 수 있도록 설계되었습니다.

### 하만카돈
1. **주요 특징**: 
   - 하만카돈은 세련된 디자인과 고품질 사운드를 강조합니다. 일반적으로 고급스러운 오디오 제품을 제공하며, 하이엔드 홈 오디오 시장을 겨냥합니다.
   - 이 브랜드는 정밀하고 균형 잡힌 사운드, 높은 해상도를 특징으로 하여 더 섬세한 음악 감상을 제공합니다.

2. **제품 라인업**:
   - **홈 오디오**: 하만카돈 Aura Studio, 하만카돈 Citation 시리즈는 뛰어난 음질과 현대적인 디자인을 갖춘 프리미엄 홈 오디오 제품입니다.
   - **사운드바**: 하만카돈 사운드바는 TV와 연계하여 풍부한 서라운드 사운드를 제공합니다.

### 차이점
- **시장 포지셔닝**: JBL은 젊은층과 대중적인 시장을 겨냥한 활기찬 브랜드로, 하만카돈은 프리미엄 시장을 대상으로 고급스러운 제품을 제공합니다.
- **사운드 프로파일**: JBL은 강력한 베이스와 생동감 있는 소리를 강조하는 반면, 하만카돈은 세밀하고 정교한 사운드 경험을 제공합니다.
- **디자인 및 기능**: JBL은 다채로운 색상과 내구성을 갖춘 디자인을 제공하며, 하만카돈은 세련된 외관과 고급스러운 마감을 특징으로 합니다.

이처럼 삼성전자가 소유한 JBL과 하만카돈은 각각의 고객 요구에 맞춰 다양한 오디오 솔루션을 제공합니다.

Original Input: 갤럭시 버즈 이어버드 한쪽을 새로 구매했는데 페어링 어떻게 하나요
Google Search Query: 갤럭시 버즈 이어버드 한쪽 페어링 방법
LLM Query: 새로 구매한 갤럭시 버즈 이어버드 한쪽을 기존에 사용 중인 이어버드와 페어링하는 방법이 궁금합니다. 이어버드 한쪽만 새롭게 연결할 수 있는 절차를 단계별로 알려주세요.
Google Search Results: 5


갤럭시 버즈 이어버드 한쪽을 기존에 사용 중인 이어버드와 페어링하는 방법은 다음과 같은 단계별 절차를 따르시면 됩니다:

### 1단계: 이어버드 준비
- 이어버드 두 개를 충전 케이스에 각각 넣고, 충전 단자에 정상적으로 안착되었는지 확인하세요.
- 충전 케이스의 LED가 녹색 또는 빨간색으로 깜빡이는지 확인합니다.

### 2단계: 커플링 설정
- 충전 케이스를 열어둔 상태에서 케이스 아래의 충전 단자 옆에 있는 연결 버튼을 7초 이상 길게 누르세요.
- 충전 케이스 LED가 파란색으로 깜빡인 후 꺼지면 커플링이 완료됩니다.

### 3단계: 문제 해결
- 만약 충전 케이스의 LED가 계속 빨간색으로 깜빡이는 경우, 단자 통신 오류일 수 있습니다. 이 경우, 1단계부터 다시 시도하세요.
- 이어버드와 케이스 단자 접점 부위에 이물질이나 부식이 있는지 확인하고, 케이스 침수 여부도 점검해 보세요.

### 추가 정보
- 새로 연결한 이어버드의 소프트웨어 버전이 기존 이어버드와 다를 경우, 페어링 모드로 진입할 수 없습니다. 두 이어버드를 케이스에 넣고 뚜껑을 닫았다가 다시 열고, 모바일 기기의 `Galaxy Wearable` 앱에서 블루투스로 직접 연결 후 소프트웨어 업데이트를 진행하세요.
- 서비스 센터 방문이 필요한 경우, 이어버드를 구입한 후 엔지니어를 통해 커플링 설정을 받을 수 있습니다.

해당 절차를 통해 새로 구매한 이어버드를 기존 이어버드와 성공적으로 페어링할 수 있습니다. 추가적인 도움이 필요하다면 서비스 센터를 방문해 보세요.

Original Input: 삼성전자 S25 무게가 S24와 비교 했을때 얼마나 차이나
Google Search Query: 삼성전자 S25 S24 무게 비교
LLM Query: 삼성전자 S25 모델과 S24 모델의 무게 차이는 얼마나 되나요? 각 모델의 정확한 무게를 알려주시고, 무게 차이가 생기는 이유도 설명해 주세요.
Google Search Results: 5


삼성전자 갤럭시 S25 시리즈와 S24 시리즈의 무게는 다음과 같습니다:

### 갤럭시 S24 시리즈
- **갤럭시 S24**: 약 163g
- **갤럭시 S24+**: 약 195g
- **갤럭시 S24 Ultra**: 약 233g

### 갤럭시 S25 시리즈
- **갤럭시 S25**: 162g
- **갤럭시 S25+**: 190g
- **갤럭시 S25 Ultra**: 218g

### 무게 차이
- **갤럭시 S25**는 갤럭시 S24보다 약 1g 가볍습니다.
- **갤럭시 S25+**는 갤럭시 S24+보다 약 5g 가볍습니다.
- **갤럭시 S25 Ultra**는 갤럭시 S24 Ultra보다 약 15g 가볍습니다.

### 무게 차이가 생기는 이유
갤럭시 S25 시리즈의 무게가 S24 시리즈보다 가벼워진 주요 이유는 디자인과 소재의 변화입니다. S25 시리즈는 더 얇아진 디자인을 채택하였으며, 특정 모델은 티타늄 소재를 사용하여 내구성을 유지하면서도 무게를 줄였습니다. 또한 내부 구성 요소의 최적화와 배터리 기술의 향상도 무게 감소에 기여하였습니다. 이러한 변화는 사용자 경험을 개선하고 더 편리한 휴대성을 제공하기 위해 이루어졌습니다.